# Benchmark of various outlier detection models


  1. Linear Models for Outlier Detection:
     1. **PCA: Principal Component Analysis** use the sum of
       weighted projected distances to the eigenvector hyperplane 
       as the outlier outlier scores)
     2. **MCD: Minimum Covariance Determinant** (use the mahalanobis distances 
       as the outlier scores)
     3. **OCSVM: One-Class Support Vector Machines**
     
  2. Proximity-Based Outlier Detection Models:
     1. **LOF: Local Outlier Factor**
     2. **CBLOF: Clustering-Based Local Outlier Factor**
     3. **kNN: k Nearest Neighbors** (use the distance to the kth nearest 
     neighbor as the outlier score)
     4. **HBOS: Histogram-based Outlier Score**
  
  4. Outlier Ensembles and Combination Frameworks
     1. **Isolation Forest**

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.xgbod import XGBOD
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [2]:
# Define data file and read X and y
import pandas as pd

csv_list_file = ['Woojoo_data_temp1.csv','Woojoo_data_temp2.csv','Woojoo_data_temp3.csv','Woojoo_data_hum1.csv',
                'Woojoo_data_hum2.csv','Woojoo_data_waterlevel.csv']
# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
             'CBLOF', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)


for csv_file in csv_list_file:
    print("\n... Processing", csv_file, '...')
    mat = pd.read_csv(os.path.join('data', csv_file))

    X = mat[mat.columns[1]] #data column
    X = X.values.reshape(-1,1)
    y = mat[mat.columns[2]].ravel()
    y = y*-0.5+0.5
    outliers_fraction = np.count_nonzero(y) / len(y)
    if outliers_fraction == 0:
        outliers_fraction = 0.08

    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    print(outliers_percentage)
    # construct containers for saving results
    roc_list = [csv_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [csv_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [csv_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    classifiers = {
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
        
    }

    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
            clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing Woojoo_data_temp1.csv ...
9.3285
Cluster-based Local Outlier Factor ROC:0.5401, precision @ rank n:0.3208, execution time: 1.704s
Histogram-base Outlier Detection (HBOS) ROC:0.6686, precision @ rank n:1.0, execution time: 1.67s
Isolation Forest ROC:1.0, precision @ rank n:1.0, execution time: 2.912s
K Nearest Neighbors (KNN) ROC:0.5027, precision @ rank n:1.0, execution time: 2.616s
Local Outlier Factor (LOF) ROC:0.5088, precision @ rank n:1.0, execution time: 0.61s
Minimum Covariance Determinant (MCD) ROC:0.9918, precision @ rank n:0.7853, execution time: 0.02s
One-class SVM (OCSVM) ROC:0.9971, precision @ rank n:0.8742, execution time: 178.1725s
Principal Component Analysis (PCA) ROC:0.651, precision @ rank n:0.5463, execution time: 0.005s

... Processing Woojoo_data_temp2.csv ...
9.5779
Cluster-based Local Outlier Factor ROC:0.7359, precision @ rank n:0.6038, execution time: 0.354s
Histogram-base Outlier Detection (HBOS) ROC:0.9692, precision @ rank n:1.0, execution 

In [3]:
print(roc_list)

['Woojoo_data_waterlevel', 59345, 1, 9.4296, 0.8769, 0.8493, 0.9985, 0.5177, 0.5349, 0.7693, 0.7449, 0.1763]


In [4]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,CBLOF,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,Woojoo_data_temp1,59345,1,9.3285,1.704,1.67,2.912,2.616,0.61,0.02,178.1725,0.005
0,Woojoo_data_temp2,59345,1,9.5779,0.354,0.003,3.448,2.308,0.536,0.012,190.222,0.003
0,Woojoo_data_temp3,59345,1,9.6874,0.34,0.003,3.132,2.195,0.604,0.009,175.7716,0.004
0,Woojoo_data_hum1,59345,1,9.9773,0.395,0.004,2.996,2.009,0.331,0.009,178.273,0.003
0,Woojoo_data_hum2,59345,1,9.9149,0.451,0.005,2.815,1.892,0.272,0.009,175.5401,0.004
0,Woojoo_data_waterlevel,59345,1,9.4296,0.276,0.004,2.391,3.09,1.522,0.009,166.038,0.003


Analyze the performance of ROC and Precision @ n

In [5]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,CBLOF,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,Woojoo_data_temp1,59345,1,9.3285,0.5401,0.6686,1.0,0.5027,0.5088,0.9918,0.9971,0.651
0,Woojoo_data_temp2,59345,1,9.5779,0.7359,0.9692,0.9981,0.5009,0.5075,0.9945,0.9969,0.7131
0,Woojoo_data_temp3,59345,1,9.6874,0.9992,0.9844,0.9995,0.5002,0.5163,0.9832,0.9982,0.773
0,Woojoo_data_hum1,59345,1,9.9773,0.751,0.8679,0.9992,0.5335,0.5386,0.9574,0.9999,0.6479
0,Woojoo_data_hum2,59345,1,9.9149,0.9358,0.9843,0.9975,0.527,0.531,0.9699,0.9988,0.6387
0,Woojoo_data_waterlevel,59345,1,9.4296,0.8769,0.8493,0.9985,0.5177,0.5349,0.7693,0.7449,0.1763


In [6]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,CBLOF,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,Woojoo_data_temp1,59345,1,9.3285,0.3208,1.0,1.0,1.0,1.0,0.7853,0.8742,0.5463
0,Woojoo_data_temp2,59345,1,9.5779,0.6038,1.0,0.9167,1.0,1.0,0.8241,0.9089,0.4235
0,Woojoo_data_temp3,59345,1,9.6874,1.0,0.8217,0.9529,1.0,1.0,0.6957,0.8959,0.5275
0,Woojoo_data_hum1,59345,1,9.9773,0.4643,1.0,0.9517,1.0,1.0,0.7127,1.0,0.3768
0,Woojoo_data_hum2,59345,1,9.9149,0.7992,1.0,0.922,1.0,1.0,0.7597,0.9198,0.3411
0,Woojoo_data_waterlevel,59345,1,9.4296,0.4534,1.0,0.9241,0.8421,0.672,0.225,0.2932,0.0
